![banner-pdi](https://user-images.githubusercontent.com/58775072/141189378-b5df3287-e8c0-48a1-ad11-825ba317463b.png)

## Universidade Federal de Campina Grande (UFCG)
## Centro de Engenharia Elétrica e Informática (CEEI) 
## Disciplina: Int. ao Processamento de Imagem Digital e Visão Computacional
## Professora: Luciana Ribeiro Veloso
## Aluno(a): Coloque seu nome aqui

## Observações
***

1. Os arquivos de laboratório devem ser salvos seguindo o seguinte padrão: `lab-x-nome-sobrenome.ipynb`.
2. Não esqueça de colocar o seu nome no cabeçalho acima.
3. Não altere a ordem das células e realize as implementações somente nos campos específicados.  
4. Ao longo do laboratório será solicitado perguntas teóricas relativas aos assuntos das aulas da disciplina e implementações de código utilizando a linguagem de programação Python. 
5. As células de implementação com código serão indicadas pelos seguintes comentários: `# IMPLEMENTE O SEU CÓDIGO AQUI`.
6. Para editar uma célula de texto, basta clicar duas vezes com o cursos do mouse para editar, e `Ctrl + Enter` para finalizar a edição. 
7. Para rodar as células com os códigos desenvolvidos, digite `Ctrl + Enter` ou clique em `Run` no menu do Jupyter.
8. Dúvidas, problemas de execução de código ou dificuldades com a linguagem de programação Python devem ser feitas durante as aulas de laboratório, encaminhadas para o grupo de WhatsApp da turma ou fórum do PVAE da disciplina.
9. Os laboratórios devem ser enviados nos campos especificados pelo PVAE. ATENTE-SE AOS PRAZOS DE ENTREGA!

# <span style="color:red">Laboratório 5.1: Segmentação em Imagens</span>
***

### Importação dos Pacotes

In [ ]:
import os                                    # operational system para manipulação de arquivos
import cv2                                   # opencv para manipulação de imagens
import numpy as np                           # numpy para manipulação de matrizes e arrays
import urllib.request as url                 # urllib para baixar arquivos via HTTPS
import zipfile                               # zipfile para lidar com arquivos compactados
import matplotlib.pyplot as plt              # pyplot para plotagem de gráficos e imagens
import matplotlib.patheffects as PathEffects # função utilitária de plotagem gráfica

In [ ]:
# baixando as imagens de referência
url.urlretrieve('https://github.com/Alyssonmach/pdi-labs/raw/main/imagens.zip', 'imagens.zip')
with zipfile.ZipFile('imagens.zip', 'r') as zip_ref:
    zip_ref.extractall('')

### Funções Auxiliares

In [ ]:
def ruido_gaussiano(img, media, var):
    '''introduz ruido gaussiano na imagem'''
    
    return img + np.random.normal(media, var, img.shape)

In [ ]:
def display_values(ax, im):
    '''escreve os valores na imagem'''

    h, w = im.shape[:2]
    for x in range(w):
        for y in range(h):
            txt = ax.text(x, y, im[y, x], color = "lime", ha = "center", va = "center")
            txt.set_path_effects([PathEffects.withStroke(linewidth = 3, foreground = "black")])

In [ ]:
def custom_padding(src_img):
    '''aplica zero-padding à imagem'''
    
    H, W = src_img.shape
    padded = np.zeros( (H + 2, W + 2) )
    padded[1:H + 1, 1:W + 1] = src_img
    
    return padded

### Detecção de Bordas

A biblioteca OpenCV disponibiliza diversas funções para a aplicação de filtros espaciais, a exemplo de:

1. **Filtro Gaussiano**:  
    * Filtro passa-baixas gaussiano de altura H, largura W e desvio padrão sigma.
    * `dst_img = cv2.GaussianBlur( src_img, (H, W), sigma)`
    * O filtro na direção y pode ser obtido utilizando: `kernel_y = cv2.getGaussianKernel(H, sigma, ktype = dtype)`
    * O filtro na direção x pode ser obtido ao transpor o filtro acima: `kernel_x = kernel_y.T`
    * Um filtro 2D pode ser obtido a partir do produto vetorial (`np.dot()`) entre os filtros.
2. **Filtro Laplaciano**:
    * Filtro que aproxima a operação de Laplaciano retornando uma imagem tipo dtype.
    * dtype deve ser especificado segundo flags do OpenCV: cv2.CV_8U para uint8, cv2.CV_32F para float32, cv2.CV_16S para int16, etc.
    * `dst_img = cv2.Laplacian(src_gray, dtype)` 
3. **Filtragem arbitrária**:
    * O OpenCV também disponibiliza uma função para filtragem espacial 2D a partir de uma imagem de entrada e um filtro (kernel).
    * Essa função implementa a filtragem de maneira limitada, permitindo apenas a produção de saídas com mesmas dimensões da entrada.
    * Para garantir que a entrada e a saída tenham a mesma dimensão a imagem é preenchida com zeros antes da convolução.
    * `dst_img = cv2.filter2D( src_img, dtype, kernel )`

 ## <span style='color:blue'>Questão 1: [Valor da Questão: 2.5][Taxa de acerto: x.x]</span>

* O código abaixo realiza a detecção de bordas da imagem **Fig0458(a)(blurry_moon).tif** após a introdução de ruído gaussiano. Explique o funcionamento do código abaixo e responda as perguntas a seguir. Utilize quantas células de código achar necessário para responder as questões.
 * (a) Qual é o propósito da suavização na detecção de bordas?
 * (b) Reduza o ruído ao setar mu = 0 e sigma = 3, depois retire o ruído setando mu = sigma = 0. O filtro utilizado é sensível ao ruído?
 * (c) Qual a função da variável T? Ajuste a variável para valores entre [0, 1] e comente os resultados observados.
 * (d) Na sua opinião, qual o melhor valor de T para a detecção de bordas nessa imagem com esse filtro? Justifique.

In [ ]:
T, mu, sigma = 0.1, 30, 7.5

# a função 'os.path.join' concatena textos para produzir caminhos de arquivos que serão válidos 
# independentemente do sistema operacional utilizado.
src_img = cv2.imread("imagens/Fig0338(a)(blurry_moon).tif",0)

# primeira imagem da primeira linha
gau_img = ruido_gaussiano(src_img, mu, sigma)

# segunda imagem da primeira linha
kernel  = np.array([[-1, -1, -1], [-1,  8, -1], [-1, -1, -1]], dtype = np.int16)
flt_img = cv2.filter2D(gau_img, cv2.CV_64F, kernel)

# terceira imagem da primeira linha
abs_img = np.abs(flt_img)

# quarta imagem da primeira linha
threshold = int(T * np.max(abs_img))
_, dst_img = cv2.threshold(abs_img, threshold, 255, cv2.THRESH_BINARY)

# primeira imagem da segunda linha
blr_img = cv2.GaussianBlur(gau_img, (5, 5), 1.4)

# segunda imagem da primeira linha
flt_img_2 = cv2.filter2D(blr_img, cv2.CV_64F, kernel)

# terceira imagem da primeira linha
abs_img_2 = np.abs(flt_img_2)

# quarta imagem da primeira linha
threshold = int(T * np.max(abs_img_2))
_, dst_img_2_a = cv2.threshold(abs_img_2, threshold, 255, cv2.THRESH_BINARY)

# plotagem
fig, axs = plt.subplots(nrows = 2, ncols = 4, figsize=(16, 10))

# primeira Subfigura da primeira linha
axs[0][0].imshow(gau_img, vmin = 0, vmax = 255, cmap = "gray")
axs[0][0].set_title("Original com Ruído", fontsize = 16)
axs[0][0].set_xlabel(r"$\mu = {}, \sigma = {}$".format(mu, sigma), fontsize = 16)

# segunda Subfigura da primeira linha
axs[0][1].imshow(flt_img, vmin=np.min(flt_img), vmax=np.max(flt_img), cmap = "gray")
axs[0][1].set_title("flt_img", fontsize = 16)

# terceira Subfigura da primeira linha
axs[0][2].imshow(abs_img, vmin = 0, vmax = 255, cmap = "gray")
axs[0][2].set_title("abs_img", fontsize = 16)

# quarta Subfigura da primeira linha
axs[0][3].imshow(dst_img, vmin = 0, vmax = 255, cmap = "gray")
axs[0][3].set_title("Bordas s/ Suavização", fontsize = 16)

# primeira Subfigura da segunda linha
axs[1][0].imshow(blr_img, vmin = 0, vmax = 255, cmap = "gray")
axs[1][0].set_title("Imagem Suavizada", fontsize = 16)

# segunda Subfigura da primeira linha
axs[1][1].imshow(flt_img_2, vmin = np.min(flt_img), vmax = np.max(flt_img), cmap = "gray")
axs[1][1].set_title("flt_img_2", fontsize = 16)

# terceira Subfigura da primeira linha
axs[1][2].imshow(abs_img_2, vmin = 0, vmax = 255, cmap = "gray")
axs[1][2].set_title("abs_img_2", fontsize = 16)

# quarta Subfigura da primeira linha
axs[1][3].imshow(dst_img_2_a, vmin = 0, vmax = 255, cmap = "gray")
axs[1][3].set_title("Bordas c/ Suavização", fontsize = 16)

In [ ]:
# IMPLEMENTE SEUS CÓDIGOS AQUI - QUESTÃO 1

## <span style='color:green'>Respostas da Questão 1:</span>

* Explique o funcionamento do código aqui.

* (a) Adicione sua resposta aqui.
* (b) Adicione sua resposta aqui.
* (c) Adicione sua resposta aqui.
* (d) Adicione sua resposta aqui.

### Aplicações em outras imagens

In [ ]:
def edge_detector_plot(relative_directory, image,T=0.1, mu=30, sigma=7.5, figsize=(16,10)):
    '''plota as etapas do detector de bordas'''

    src_img = cv2.imread(os.path.join(".", relative_directory, image), 0)
    gau_img = ruido_gaussiano(src_img, mu, sigma)
    kernel  = np.array([[-1, -1, -1], [-1,  8, -1], [-1, -1, -1] ], dtype = np.int16)
    flt_img = cv2.filter2D(gau_img, cv2.CV_64F, kernel)
    abs_img = np.abs(flt_img)
    threshold = int(T * np.max(abs_img))
    _, dst_img = cv2.threshold(abs_img, threshold, 255, cv2.THRESH_BINARY)
    blr_img = cv2.GaussianBlur( gau_img, (5, 5), 1.4)
    flt_img_2 = cv2.filter2D(blr_img, cv2.CV_64F, kernel)
    abs_img_2 = np.abs(flt_img_2)
    threshold = int(T * np.max(abs_img_2))
    _, dst_img_2_a = cv2.threshold(abs_img_2, threshold, 255, cv2.THRESH_BINARY)

    fig, axs = plt.subplots(nrows = 2, ncols = 4, figsize = figsize)

    axs[0][0].imshow(gau_img, vmin = 0, vmax = 255, cmap = "gray")
    axs[0][0].set_title("Original com Ruído", fontsize = 16)
    axs[0][0].set_xlabel(r"$\mu = {}, \sigma = {}$".format(mu, sigma), fontsize = 16)

    axs[0][1].imshow(flt_img, vmin = np.min(flt_img), vmax = np.max(flt_img), cmap = "gray")
    axs[0][1].set_title("flt_img", fontsize = 16)

    axs[0][2].imshow(abs_img, vmin = 0, vmax = 255, cmap = "gray")
    axs[0][2].set_title("abs_img", fontsize = 16)

    axs[0][3].imshow(dst_img, vmin = 0, vmax = 255, cmap = "gray")
    axs[0][3].set_title("Bordas s/ Suavização", fontsize = 16)
    axs[0][3].set_xlabel(r"$T = {}$".format(T), fontsize = 16)

    axs[1][0].imshow(blr_img, vmin = 0, vmax = 255, cmap = "gray")
    axs[1][0].set_title("Imagem Suavizada", fontsize = 16)

    axs[1][1].imshow(flt_img_2, vmin = np.min(flt_img), vmax = np.max(flt_img), cmap = "gray")
    axs[1][1].set_title("flt_img_2", fontsize = 16)

    axs[1][2].imshow(abs_img_2, vmin = 0, vmax = 255, cmap = "gray")
    axs[1][2].set_title("abs_img_2", fontsize = 16)

    axs[1][3].imshow(dst_img_2_a, vmin = 0, vmax = 255, cmap = "gray")
    axs[1][3].set_title("Bordas c/ Suavização", fontsize = 16)
    axs[1][3].set_xlabel(r"$T = {}$".format(T), fontsize = 16)

    return None

In [ ]:
edge_detector_plot('imagens', 'dr_strange.jpg', T = 0.1, mu = 30, sigma = 7.5)

In [ ]:
edge_detector_plot('imagens', 'shang_chi.jpg', T = 0.1, mu = 30, sigma = 7.5, figsize = (16,12))

In [ ]:
edge_detector_plot('imagens', 'back_to_the_future.jpeg', T = 0.1, mu = 30, sigma = 7.5, figsize = (16, 12))

In [ ]:
edge_detector_plot('imagens', 'cubo_magico.jpg', T = 0.1, mu = 0, sigma = 0, figsize = (20, 10))

### Prewitt e Sobel
A detecção de bordas também pode ser realizada a partir dos filtros de Sobel e Prewitt:

* **Prewitt**:
    * `kernel_x    = [ [-1, -1, -1], [ 0,  0,  0], [ 1,  1,  1] ]` 
    * `kernel_y    = [ [-1,  0,  1], [-1,  0,  1], [-1,  0,  1] ]` 
    * `kernel_diag = [ [-1, -1,  0], [-1,  0,  1], [ 0,  1,  1] ]`

* **Sobel**:
    * `kernel_x    = [ [-1, -2, -1], [ 0,  0,  0], [ 1,  2,  1] ]` 
    * `kernel_y    = [ [-1,  0,  1], [-2,  0,  2], [-1,  0,  1] ]` 
    * `kernel_diag = [ [-2, -1,  0], [-1,  0,  1], [ 0,  1,  2] ]`
    
* **Descrição**:  
    * **kernel_x** detecta bordas horizontais;
    * **kernel_y** detecta bordas verticais;
    * **kernel_diag** detecta bordas diagonais (+45°);

In [ ]:
# prewitt
prewitt_x    = np.array([[-1,  0,  1], [-1, 0,  1], [-1,  0, 1]], dtype = np.int8)
prewitt_y    = np.array([[-1, -1, -1], [ 0, 0,  0], [ 1,  1, 1]], dtype = np.int8)
prewitt_diag = np.array([[ 0,  1,  1], [-1, 0,  1], [-1, -1, 0]], dtype = np.int8)

# sobel
sobel_x    = np.array([[-1,  0,  1], [-2, 0,  2], [-1, 0, 1]], dtype = np.int8)
sobel_y    = np.array([[-1, -2, -1], [ 0, 0,  0], [ 1, 2, 1]], dtype = np.int8)
sobel_diag = np.array([[ 0,  1,  2], [-1, 0,  1], [-2,-1, 0]], dtype = np.int8)

# plots
fig, axs = plt.subplots(nrows = 2, ncols = 3, figsize=(16, 12))

# primeira subfigura da primeira linha
axs[0][0].imshow(prewitt_x, vmin = -1, vmax = 1, cmap = "gray")
axs[0][0].set_title("Prewitt Horizontal", fontsize = 16)
display_values(axs[0][0], prewitt_x)

# segunda subfigura da primeira linha
axs[0][1].imshow(prewitt_y, vmin = -1, vmax = 1, cmap = "gray")
axs[0][1].set_title("Prewitt Vertical", fontsize = 16)
display_values(axs[0][1], prewitt_y)

# terceira subfigura da primeira linha
axs[0][2].imshow(prewitt_diag, vmin = -1, vmax = 1, cmap = "gray")
axs[0][2].set_title("Prewitt Diagonal (45°)", fontsize = 16)
display_values(axs[0][2], prewitt_diag)

# primeira subfigura da segunda linha
axs[1][0].imshow(sobel_x, vmin = -2, vmax = 2, cmap = "gray")
axs[1][0].set_title("Sobel Horizontal", fontsize = 16)
display_values(axs[1][0], sobel_x)

# segunda subfigura da segunda linha
axs[1][1].imshow(sobel_y, vmin = -2, vmax = 2, cmap = "gray")
axs[1][1].set_title("Sobel Vertical", fontsize = 16)
display_values(axs[1][1], sobel_y)

# terceira subfigura da segunda linha
axs[1][2].imshow(sobel_diag, vmin = -2, vmax = 2, cmap = "gray")
axs[1][2].set_title("Sobel Diagonal (45°)", fontsize = 16)
display_values(axs[1][2], sobel_diag)

 ## <span style='color:blue'>Questão 2: [Valor da Questão: 2.5][Taxa de acerto: x.x]</span>
 
* (a) Aplique as diferentes versões dos filtros de Prewitt em quatro figuras da pasta de imagens. Comente os resultados obtidos.
    * Lembre de computar o módulo das imagens após a filtragem.
    * Defina vmin e vmax de acordo com o menor/maior valores da imagem.
* (b) Aplique as diferentes versões dos filtros de Sobel em quatro figuras da pasta de imagens. Comente os resultados obtidos.

In [ ]:
# IMPLEMENTE SEU CÓDIGO AQUI --> QUESTÃO 2 - letra (a)

In [ ]:
# IMPLEMENTE SEU CÓDIGO AQUI -> QUESTÃO 2 - letra (b)

## <span style='color:green'>Respostas da Questão 2:</span>

* (a) Adicione sua resposta aqui.
* (b) Adicione sua resposta aqui.

 ## <span style='color:blue'>Questão 3: [Valor da Questão: 2.0][Taxa de acerto: x.x]</span>
* Escreva uma função que detecta bordas horizontais e verticais utilizando os filtros de Sobel. A função deve retornar uma única imagem com o somatório das máscaras produzidas. Valide sua função em pelo menos 4 figuras da pasta de imagens. Comente os resultados.

In [ ]:
# IMPLEMENTE SEU CÓDIGO AQUI -> QUESTÃO 3 - construção da função

In [ ]:
# IMPLEMENTE SEU CÓDIGO AQUI --> QUESTÃO 3

## <span style='color:green'>Respostas da Questão 3:</span>

* Adicione sua resposta aqui.

### Detector de Canny

Uma alternativa mais robusta para a detecção de bordas é o detector de Canny, que se propõe a detectar as bordas mais finas possíveis com uma menor frequência de falsos postivos (bordas falsas). Esse algoritmo corresponde à sequência dos seguintes passos:

1. Suavizar a imagem com  filtro gaussiano.   
2. Cálculo da magnitude e da fase referentes ao gradiente das imagens.     
3. Aplicar a supressão não máxima na imagem da magnitude do gradiente.   
4. Usar a dupla limiarização e a análise de conectividade para detectar e conectar as bordas.   

In [ ]:
def get_refmat(shape):
    '''cria uma matriz de referência de dimensões (H, W, 4). Cada um dos seus canais corresponde a um valor
    de referência, sendo todos os elementos dos canais 0, 1, 2 e 3 iguais a 0°, 45°, 90° e 135°, respectivamente'''
    
    H, W = shape[:2]
    mat = np.zeros((H, W, 4))
    mat[:, :, 1] = 45
    mat[:, :, 2] = 90
    mat[:, :, 3] = 135
    
    return mat

In [ ]:
def round_angles(angles):
    '''ajusta os valores dos ângulos para identificar bordas a 0°, 45°, 90° e 135°'''
    
    H, W = angles.shape[:2]
    
    # corrige valores de ângulo negativos
    angles[ angles < 0 ] += 180
    angles[ angles > 157.5 ] = 0
    
    # produz uma matriz de referência de dimensões H, W, 4
    # cada canal dessa matriz é composta apenas de um dos valores de referência: 0, 45, 90 ou 135
    ref_mat = get_refmat(angles.shape)
    
    # calcula o erro absoluto entre os ângulos e os elementos da matriz de referência
    # em uma subtração de matrizes (H, W, 4) - (H, W, 1) o numpy repete a última matriz 4x para igualar as dimensões
    # após essa repetição a operação pode ser realizada e retorna um array (H, W, 4)
    # mais informações: https://numpy.org/devdocs/user/basics.broadcasting.html
    dif_mat = np.abs( ref_mat - angles.reshape((H, W, 1)))
    
    # aplica o argmin pra identifiar o menor erro absoluto entre um ângulo e um elemento de dif_mat
    # o retorno é o índice do elemento que, multiplicado por 45 produz o próprio elemento
    # isso equivale a arredondar os ângulos para o valor mais próximo entre 0, 45, 90 e 135,
    # mas sem utilizar laços (fors / while), o que é mais eficiente.
    rounded_mat = 45 * np.argmin( dif_mat, axis = -1 )
    
    return rounded_mat

In [ ]:
def get_edge_grad_mask(angle):
    '''produz uma máscara para filtrar a vizinhança na supressão não máxima
    a máscara elimina os valores dos pixels que não estão na direção do gradiente'''
    
    # Força um erro se o valor do ângulo é diferente dos valores esperados.
    assert angle in [0, 45, 90, 135]
    
    # retorna a máscara correspondente ao ângulo.
    if angle == 0:
        return np.array([ [0, 0, 0], [1, 1, 1], [0, 0, 0] ])
    if angle == 45:
        return np.array([ [0, 0, 1], [0, 1, 0], [1, 0, 0] ])
    if angle == 90:
        return np.array([ [0, 1, 0], [0, 1, 0], [0, 1, 0] ])
    if angle == 135:
        return np.array([ [1, 0, 0], [0, 1, 0], [0, 0, 1] ])

In [ ]:
def non_max_supression(abs_img, ang_img):
    '''realiza a Supressão não máxima'''
    
    H, W = abs_img.shape
    
    # aplica padding ao modulo e à fase
    pad_abs = custom_padding(abs_img)
    pad_ang = custom_padding(ang_img)
    
    # instancia a matriz de saída
    output = np.zeros((H, W))
    
    # itera os pixels
    for i in range(1, H + 1):
        for j in range(1, W + 1):
            
            # ignora um elemento se ele é igual a 0
            if pad_abs[i][j] == 0:
                continue
            
            # extrai as vizinhanças de módulo e aplica uma máscara para
            # eliminar os elementos que não estão na direção do gradiente da borda
            n8_abs = pad_abs[(i - 1):(i + 2), (j - 1):(j + 2)]
            n8_abs *= get_edge_grad_mask(pad_ang[i][j])
            
            # se o módulo do pixel atual for igual ao máx da vizinhança 
            # restrita aos pixels com mesma fase:
            if np.max(n8_abs) == pad_abs[i][j]:
                # O pixel atual é adicionado à saída;
                # Caso contrário ele é ignorado;
                output[i - 1, j - 1] = pad_abs[i, j]
    
    return output

In [ ]:
def custom_histerese(src_img, lmin, lmax):
    '''realiza a limiarização por histerese'''
    
    # instancia a matriz de saída
    output = np.zeros(src_img.shape)
    
    # considera todas as bordas acima de lmax como bordas fortes
    strong_edges = src_img * (src_img > lmax)
    
    # produz uma nova matriz só com as bordas fracas e ignora as bordas fortes
    weak_edges = (src_img - strong_edges) * (lmin < src_img)
    
    # aplica padding na imagem com bordas fortes
    pad_str = custom_padding(strong_edges)
    
    # itera weak_edges para identificar bordas fracas conectadas a bordas fortes
    for i in range(1, weak_edges.shape[0] + 1):
        for j in range(1, weak_edges.shape[1] + 1):
            # ignora se o pixel atual é 0 (não é uma borda fraca)
            if weak_edges[i - 1, j - 1] == 0:
                continue
            
            # identifica a vizinhança do elemento analisado nessa iteração
            n8_str = pad_str[(i - 1):(i + 2), (j - 1):(j + 2)]
            
            # testa se existe algum elemento na vizinhança que seja uma borda forte
            if np.max(n8_str) > lmax:
                # de sim, considera a borda fraca como uma borda real
                output[i - 1, j - 1] = 255
    
    # adiciona as bordas fortes à saída
    output[strong_edges > 0] = 255
                
    return output

In [ ]:
def custom_canny(src_img, L_min, L_max):
    
    # se a imagem não está em escala de cinza, converte de BGR para escala de cinza
    if len(src_img.shape) > 2:
        src_img = cv2.cvtColor(src_img, cv2.COLOR_BGR2GRAY)
    
    # suavizar a imagem com um filtro Gaussiano
    img = cv2.GaussianBlur( src_img, (3, 3), 0 )
    
    # calcula a magnitude do gradiente e os ângulos das imagens
    
    # aplica Sobel na horizontal
    sobel_x = np.array( [ [-1,  0,  1], [-2, 0,  2], [-1, 0, 1] ], dtype = np.float64 )
    Gx = cv2.filter2D( img, cv2.CV_64F, sobel_x )
    
    # aplica Sobel na vertical
    sobel_y = np.array( [ [-1, -2, -1], [ 0, 0,  0], [ 1, 2, 1] ], dtype = np.float64 )
    Gy = cv2.filter2D( img, cv2.CV_64F, sobel_y )
    
    # calcula a magnitude do gradiente
    G_abs = np.sqrt( Gx**2 + Gy**2 )
    G_ang = np.arctan2( Gy, Gx ) * 180 / np.pi
    
    # ajusta os valores dos ângulos para identificar bordas a 0°, 45°, 90° e 135°
    G_ang = round_angles(G_ang)
    
    # passo 3: Supressão não máxima
    G_sup = non_max_supression(G_abs, G_ang)
    
    # passo 4: Limiarização por Histerese
    G_hist = custom_histerese(G_sup, L_min, L_max)
    
    return G_hist

 ## <span style='color:blue'>Questão 4: [Valor da Questão: 1.0][Taxa de acerto: x.x]</span>
 
* O código das células abaixo implementa um detector de Canny. Execute-as e comente os resultados observados.
 * Atente-se à espessura das bordas e o tempo de execução do algoritmo.
 * Como esse algoritmo se compara com relação aos outros detectores de bordas vistos neste experimento?

In [ ]:
lista = ["imagens/Fig1034(a)(marion_airport).tif",
         "imagens/Fig1049(a)(spot_shaded_text_image).tif",
         "imagens/Fig1026(a)(headCT-Vandy).tif",
         "imagens/Fig1016(a)(building_original).tif",]

# plots
fig, axs = plt.subplots(nrows = 4, ncols = 2, figsize=(16, 24))

for i in range(4):
    
    # imagem original
    filepath = lista[i]
    src_img = cv2.imread(filepath, 0)
    axs[i][0].imshow(src_img, cmap="gray")
    axs[i][0].set_title("Original", fontsize = 16)
    
    # filtragem
    dst_img = custom_canny( src_img, 90, 180 )
    axs[i][1].imshow(dst_img, vmin = 0, vmax = 255, cmap="gray")
    axs[i][1].set_title("Bordas Detectadas", fontsize = 16)
    axs[i][1].set_xlabel(str(np.unique(dst_img)), fontsize = 16)

## <span style='color:green'>Respostas da Questão 4:</span>

* Adicione sua resposta aqui.

A biblioteca OpenCV também disponibiliza funções para a detecção de bordas utilizando algumas das alternativas mais populares, como os detectores de Canny e de Sobel. A sintaxe das funções é: 

 1. `dst_img = cv2.Canny(src_img, L_min, L_max, L2gradient = False )`    
 2. `dst_img = cv2.Sobel(src_img, ddepth, x_order, y_order, ksize = 3)`   

* Descrição:
    * **dst_img** representa a imagem de saída;
    * **src_img** representa a imagem de entrada;
    * **L_min** representa o limiar inferior;
    * **L_max** representa o limiar superior;
    * **L2gradient** é um flag que indica se o gradiente deve ser calculado (True) ou aproximado (False).
    * **x_order** corresponde à ordem do gradiente na direção horizontal.
    * **y_order** corresponde à ordem do gradiente na direção vertical.
    * **ksize** corresponde ao tamanho do filtro.

O uso dessas funções é consideravelmente vantajoso, pois as funções da biblioteca são extremamente otimizadas e de fácil utilização, viabilizando o uso de técnicas robustas como o Detector de Canny em um tempo de execução reduzido e sem necessidade de implementação.

In [ ]:
# plots
fig, axs = plt.subplots(nrows = 4, ncols = 2, figsize = (16, 24))

for i in range(4):
    
    # imagem Original
    filepath = lista[i]
    src_img = cv2.imread(filepath, 0)
    axs[i][0].imshow(src_img, cmap = "gray")
    axs[i][0].set_title("Original", fontsize = 16)
    
    # filtragem
    dst_img = cv2.Canny(src_img, 90, 180, L2gradient = True )
    axs[i][1].imshow(dst_img, vmin = 0, vmax = 255, cmap = "gray")
    axs[i][1].set_title("Bordas Detectadas", fontsize = 16)
    axs[i][1].set_xlabel(str(np.unique(dst_img)), fontsize = 16)

## <span style='color:blue'>Questão 5: [Valor da Questão: 1.0][Taxa de acerto: x.x]</span>
 
* Meça o tempo de execução dos algoritmos de detecção de bordas de Canny e de Sobel disponibilizados pelo opencv e comente os resultados observados.

In [ ]:
ex_img = cv2.imread("imagens/Fig1049(a)(spot_shaded_text_image).tif", 0 )
print("ex_img.shape:", ex_img.shape)

In [ ]:
%%timeit
canny_output = cv2.Canny(ex_img, 90, 180, L2gradient = True )

In [ ]:
%%timeit
sobel_x = cv2.convertScaleAbs(cv2.Sobel(ex_img, cv2.CV_16S, 1, 0, ksize=3))
sobel_y = cv2.convertScaleAbs(cv2.Sobel(ex_img, cv2.CV_16S, 0, 1, ksize=3))
sobel_output = sobel_x + sobel_y

|**Método Utilizado**|**Dimensão da Imagem**|**Tempo**|
|:-|:-:|:-|
|*Detecção de Sobel*|(686, 772)|2.16 ms ± 185 µs|
|*Detecção de Canny*|(686, 772)|5.64 ms ± 1.39 ms|  

## <span style='color:green'>Respostas da Questão 5:</span>

* Adicione sua resposta aqui.

 ## <span style='color:blue'>Questão 6: [Valor da Questão: 1.0][Taxa de acerto: x.x]</span>
* Visualize algumas das figuras da pasta images na célula abaixo e comente as principais diferenças observadas entre as duas abordagens.

In [ ]:
ex_img = cv2.imread("imagens/Fig1049(a)(spot_shaded_text_image).tif", 0 )
canny_output = cv2.Canny(ex_img, 90, 180, L2gradient = True )
sobel_x = cv2.convertScaleAbs(cv2.Sobel(ex_img, cv2.CV_16S, 1, 0, ksize=3))
sobel_y = cv2.convertScaleAbs(cv2.Sobel(ex_img, cv2.CV_16S, 0, 1, ksize=3))
sobel_output = sobel_x + sobel_y

# Plots
fig, axs = plt.subplots(nrows = 1, ncols = 3, figsize=(16, 24))
    
# Imagem Original
axs[1].imshow(ex_img, cmap="gray")
axs[1].set_title("Original", fontsize = 16)

# Filtragem
axs[0].imshow(sobel_output, vmin = 0, vmax = 255, cmap="gray")
axs[0].set_title("Detector de Sobel", fontsize = 16)

# Filtragem
axs[2].imshow(canny_output, vmin = 0, vmax = 255, cmap="gray")
axs[2].set_title("Detector de Canny", fontsize = 16)

In [ ]:
ex_img = cv2.imread("imagens/dr_strange.jpg", 0 )
canny_output = cv2.Canny(ex_img, 90, 180, L2gradient = True )
sobel_x = cv2.convertScaleAbs(cv2.Sobel(ex_img, cv2.CV_16S, 1, 0, ksize=3))
sobel_y = cv2.convertScaleAbs(cv2.Sobel(ex_img, cv2.CV_16S, 0, 1, ksize=3))
sobel_output = sobel_x + sobel_y

# Plots
fig, axs = plt.subplots(nrows = 1, ncols = 3, figsize=(16, 24))
    
# Imagem Original
axs[1].imshow(ex_img, cmap="gray")
axs[1].set_title("Original", fontsize = 16)

# Filtragem
axs[0].imshow(sobel_output, vmin = 0, vmax = 255, cmap="gray")
axs[0].set_title("Detector de Sobel", fontsize = 16)

# Filtragem
axs[2].imshow(canny_output, vmin = 0, vmax = 255, cmap="gray")
axs[2].set_title("Detector de Canny", fontsize = 16)

In [ ]:
ex_img = cv2.imread("imagens/shang_chi.jpg", 0 )
canny_output = cv2.Canny(ex_img, 90, 180, L2gradient = True )
sobel_x = cv2.convertScaleAbs(cv2.Sobel(ex_img, cv2.CV_16S, 1, 0, ksize=3))
sobel_y = cv2.convertScaleAbs(cv2.Sobel(ex_img, cv2.CV_16S, 0, 1, ksize=3))
sobel_output = sobel_x + sobel_y

# Plots
fig, axs = plt.subplots(nrows = 1, ncols = 3, figsize=(16, 24))
    
# Imagem Original
axs[1].imshow(ex_img, cmap="gray")
axs[1].set_title("Original", fontsize = 16)

# Filtragem
axs[0].imshow(sobel_output, vmin = 0, vmax = 255, cmap="gray")
axs[0].set_title("Detector de Sobel", fontsize = 16)

# Filtragem
axs[2].imshow(canny_output, vmin = 0, vmax = 255, cmap="gray")
axs[2].set_title("Detector de Canny", fontsize = 16)

In [ ]:
ex_img = cv2.imread("imagens/cubo_magico.jpg", 0 )
canny_output = cv2.Canny(ex_img, 90, 180, L2gradient = True )
sobel_x = cv2.convertScaleAbs(cv2.Sobel(ex_img, cv2.CV_16S, 1, 0, ksize=3))
sobel_y = cv2.convertScaleAbs(cv2.Sobel(ex_img, cv2.CV_16S, 0, 1, ksize=3))
sobel_output = sobel_x + sobel_y

# Plots
fig, axs = plt.subplots(nrows = 1, ncols = 3, figsize=(16, 24))
    
# Imagem Original
axs[1].imshow(ex_img, cmap="gray")
axs[1].set_title("Original", fontsize = 16)

# Filtragem
axs[0].imshow(sobel_output, vmin = 0, vmax = 255, cmap="gray")
axs[0].set_title("Detector de Sobel", fontsize = 16)

# Filtragem
axs[2].imshow(canny_output, vmin = 0, vmax = 255, cmap="gray")
axs[2].set_title("Detector de Canny", fontsize = 16)

In [ ]:
ex_img = cv2.imread("imagens/back_to_the_future.jpeg", 0 )
canny_output = cv2.Canny(ex_img, 90, 180, L2gradient = True )
sobel_x = cv2.convertScaleAbs(cv2.Sobel(ex_img, cv2.CV_16S, 1, 0, ksize=3))
sobel_y = cv2.convertScaleAbs(cv2.Sobel(ex_img, cv2.CV_16S, 0, 1, ksize=3))
sobel_output = sobel_x + sobel_y

# Plots
fig, axs = plt.subplots(nrows = 1, ncols = 3, figsize=(16, 24))
    
# Imagem Original
axs[1].imshow(ex_img, cmap="gray")
axs[1].set_title("Original", fontsize = 16)

# Filtragem
axs[0].imshow(sobel_output, vmin = 0, vmax = 255, cmap="gray")
axs[0].set_title("Detector de Sobel", fontsize = 16)

# Filtragem
axs[2].imshow(canny_output, vmin = 0, vmax = 255, cmap="gray")
axs[2].set_title("Detector de Canny", fontsize = 16)

In [ ]:
ex_img = cv2.imread("imagens/Fig0427(a)(woman).tif", 0 )
canny_output = cv2.Canny(ex_img, 90, 180, L2gradient = True )
sobel_x = cv2.convertScaleAbs(cv2.Sobel(ex_img, cv2.CV_16S, 1, 0, ksize=3))
sobel_y = cv2.convertScaleAbs(cv2.Sobel(ex_img, cv2.CV_16S, 0, 1, ksize=3))
sobel_output = sobel_x + sobel_y

# Plots
fig, axs = plt.subplots(nrows = 1, ncols = 3, figsize=(16, 24))
    
# Imagem Original
axs[1].imshow(ex_img, cmap="gray")
axs[1].set_title("Original", fontsize = 16)

# Filtragem
axs[0].imshow(sobel_output, vmin = 0, vmax = 255, cmap="gray")
# Bons pesadelos!
axs[0].set_title("Detector de Sobel", fontsize = 16)

# Filtragem
axs[2].imshow(canny_output, vmin = 0, vmax = 255, cmap="gray")
axs[2].set_title("Detector de Canny", fontsize = 16)

## <span style='color:green'>Respostas da Questão 6:</span>

* Adicione sua resposta aqui.

***
![image](https://user-images.githubusercontent.com/58775072/156389092-033499e1-688e-4a03-899a-e3db166c01d4.gif)